In [1]:
import os
import sys

import pytest

import numpy as np
import scipy.io
import xarray as xr

sys.path.append('../')
import pyBLP

In [2]:
class Data(object):
    ''' Synthetic data for Nevo (2000b)

    http://www.rasmusen.org/zg604/lectures/blp/frontpage.htm

    '''
    def __init__(self):
        try:
            ps2 = scipy.io.loadmat('examples/ps2.mat')
            Z_org = scipy.io.loadmat('examples/iv.mat')['iv']
        except:
            ps2 = scipy.io.loadmat('ps2.mat')
            Z_org = scipy.io.loadmat('iv.mat')['iv']

        nsiminds = self.nsiminds = 20  # number of simulated "indviduals" per market
        nmkts = self.nmkts = 94  # number of markets = (# of cities) * (# of quarters)
        nbrands = self.nbrands = 24  # number of brands per market

        ids = ps2['id'].reshape(-1, )
        self.ids = xr.DataArray(
            ids.reshape((nmkts, nbrands), order='F'),
            coords=[range(nmkts), range(nbrands)],
            dims=['markets', 'brands'],
            attrs={'Desc': 'an id variable in the format bbbbccyyq, where bbbb '
                           'is a unique 4 digit identifier for each brand (the '
                           'first digit is company and last 3 are brand, i.e., '
                           '1006 is K Raisin Bran and 3006 is Post Raisin Bran), '
                           'cc is a city code, yy is year (=88 for all observations '
                           'in this data set) and q is quarter.'}
            )

        X1 = np.array(ps2['x1'].todense())

        self.X1 = xr.DataArray(
            X1.reshape(nmkts, nbrands, -1),
            coords=[range(nmkts), range(nbrands),
                    ['Price'] + ['Brand_{}'.format(brand)
                                 for brand in range(nbrands)]],
            dims=['markets', 'brands', 'vars'],
            attrs={'Desc': 'the variables that enter the linear part of the '
                           'estimation. Here this consists of a price variable '
                           '(first column) and 24 brand dummy variables.'}
            )
            
        X2 = np.array(ps2['x2'].copy())
        self.X2 = xr.DataArray(
            X2.reshape(nmkts, nbrands, -1),
            coords=[range(nmkts), range(nbrands),
                    ['Constant', 'Price', 'Sugar', 'Mushy']],
            dims=['markets', 'brands', 'vars'],
            attrs={'Desc': 'the variables that enter the non-linear part of the '
                           'estimation.'}
            )
            
        self.id_demo = ps2['id_demo'].reshape(-1, )

        D = np.array(ps2['demogr'])
        self.D = xr.DataArray(
            D.reshape((nmkts, nsiminds, -1), order='F'),
            coords=[range(nmkts), range(nsiminds),
                    ['Income', 'Income^2', 'Age', 'Child']],
            dims=['markets', 'nsiminds', 'vars'],
            attrs={'Desc': 'Demeaned draws of demographic variables from the CPS for 20 '
                           'individuals in each market.',
                   'Child': 'Child dummy variable (=1 if age <= 16)'}
            )

        v = np.array(ps2['v'])
        self.v = xr.DataArray(
            v.reshape((nmkts, nsiminds, -1), order='F'),
            coords=[range(nmkts), range(nsiminds),
                    ['Constant', 'Price', 'Sugar', 'Mushy']],
            dims=['markets', 'nsiminds', 'vars'],
            attrs={'Desc': 'random draws given for the estimation.'}
            )

        s_jt = ps2['s_jt'].reshape(-1, )  # s_jt for nmkts * nbransd
        self.s_jt = xr.DataArray(
            s_jt.reshape((nmkts, nbrands)),
            coords=[range(nmkts), range(nbrands),],
            dims=['markets', 'brands'],
            attrs={'Desc': 'Market share of each brand.'}
            )

        self.ans = ps2['ans'].reshape(-1, )

        Z = np.c_[Z_org[:, 1:], X1[:, 1:]]
        self.Z = xr.DataArray(
            Z.reshape((self.nmkts, self.nbrands, -1)),
            coords=[range(nmkts), range(nbrands), range(Z.shape[-1])],
            dims=['markets', 'brands', 'vars'],
            attrs={'Desc': 'Instruments'}
            )

In [6]:
if __name__ == '__main__':
    """ Load data and evaluate the 
    """
    data = Data()

    BLP = pyBLP.BLP(data)

    θ20 = np.array([[ 0.3772,  3.0888,      0,  1.1859,       0],
                    [ 1.8480, 16.5980, -.6590,       0, 11.6245],
                    [-0.0035, -0.1925,      0,  0.0296,       0],
                    [ 0.0810,  1.4684,      0, -1.5143,       0]])

    BLP.estimate(θ20=θ20, method='Nelder-Mead', maxiter=1)

    results = BLP.results

    # Run the line below to get true estimation results
    # BLP.estimate(θ20=θ20)

contraction mapping finished in 129 iterations
GMM value: 14.900789413856307
contraction mapping finished in 0 iterations
GMM value: 14.90078941385608
contraction mapping finished in 102 iterations
GMM value: 14.926139916893748
contraction mapping finished in 107 iterations
GMM value: 14.911646850460107


/Users/zhuonijie/py3_kernel/lib/python3.5/site-packages/scipy/optimize/_minimize.py:415: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  RuntimeWarning)


contraction mapping finished in 105 iterations
GMM value: 14.900993872378312
contraction mapping finished in 97 iterations
GMM value: 14.901100865528303
contraction mapping finished in 120 iterations
GMM value: 15.390375098808585
contraction mapping finished in 114 iterations
GMM value: 15.186885675491162
contraction mapping finished in 111 iterations
GMM value: 15.21888649190931
contraction mapping finished in 113 iterations
GMM value: 14.962457469270932
contraction mapping finished in 112 iterations
GMM value: 15.05705191010706
contraction mapping finished in 115 iterations
GMM value: 15.100190754967374
contraction mapping finished in 112 iterations
GMM value: 14.911902228515883
contraction mapping finished in 109 iterations
GMM value: 15.093411532237605
contraction mapping finished in 109 iterations
GMM value: 14.947333588659317
contraction mapping finished in 110 iterations
contraction mapping finished in 0 iterations
contraction mapping finished in 0 iterations
               Mean

In [12]:
θ20 = np.array([[ 0.5,  3.0888,      0,  1.1859,       0],
                [ 1.8480, 16.5980, -.6590,       0, 11.6245],
                [-0.0035, -0.1925,      0,  0.0296,       0],
                [ 0.0810,  1.4684,      0, -1.5143,       0]])

BLP.estimate(θ20=θ20)

contraction mapping finished in 134 iterations
GMM value: 15.99589803300659
contraction mapping finished in 0 iterations
contraction mapping finished in 0 iterations
GMM value: 15.995898033006595
contraction mapping finished in 2078 iterations
GMM value: 3155.947884994534
contraction mapping finished in 0 iterations
contraction mapping finished in 135 iterations
GMM value: 16.165395570671887
contraction mapping finished in 0 iterations
contraction mapping finished in 112 iterations
GMM value: 15.787405524264736
contraction mapping finished in 0 iterations
contraction mapping finished in 120 iterations
GMM value: 16.238843474461248
contraction mapping finished in 0 iterations
contraction mapping finished in 115 iterations
GMM value: 15.735280618138695
contraction mapping finished in 0 iterations
contraction mapping finished in 107 iterations
GMM value: 15.638326684403596
contraction mapping finished in 0 iterations
contraction mapping finished in 113 iterations
GMM value: 15.46237275714

In [7]:
BLP.results

{'GMM': 4.561514655034235,
 'varcov': array([[ 2.19135162e+02, -6.78231146e+00, -9.80159043e+00, ...,
          1.93077787e-01,  1.75811490e+00, -1.14549988e+01],
        [-6.78231146e+00,  7.37666990e-01,  4.18557102e-01, ...,
         -2.76519432e-03,  1.13276716e-02, -3.76281988e-01],
        [-9.80159043e+00,  4.18557102e-01,  4.98238012e-01, ...,
         -8.05583949e-03, -4.38886037e-02,  3.06141176e-01],
        ...,
        [ 1.93077787e-01, -2.76519432e-03, -8.05583949e-03, ...,
          6.75235417e-04,  2.16678789e-03, -2.16966249e-02],
        [ 1.75811490e+00,  1.13276716e-02, -4.38886037e-02, ...,
          2.16678789e-03,  4.45033735e-01, -1.28546589e+00],
        [-1.14549988e+01, -3.76281988e-01,  3.06141176e-01, ...,
         -2.16966249e-02, -1.28546589e+00,  1.69955296e+01]]),
 'β': {'Chisq': 137779.34317473238,
  'Rsq': 0.45910433361278524,
  'Rsq_G': 0.10116438381197235,
  'se': array([ 0.32699746, 14.80321459,  0.01603637,  0.19858242]),
  'β': array([ -2.0099190